In [1]:
import ray
ray.init()

 

2022-04-29 18:33:30,623	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8267


RayContext(dashboard_url='127.0.0.1:8267', python_version='3.8.10', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '192.168.2.133', 'raylet_ip_address': '192.168.2.133', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-04-29_18-33-25_947795_47234/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-04-29_18-33-25_947795_47234/sockets/raylet', 'webui_url': '127.0.0.1:8267', 'session_dir': '/tmp/ray/session_2022-04-29_18-33-25_947795_47234', 'metrics_export_port': 62329, 'gcs_address': '192.168.2.133:59213', 'address': '192.168.2.133:59213', 'node_id': '801ee2cdaf9b83a8e44f7a36e74df3dcb39e8f08796cc3e5b19cab07'})

In [2]:
items = [{"name": str(i), "data": i} for i in range(10000)]
ds = ray.data.from_items(items)   
ds.show(5) 

{'name': '0', 'data': 0}
{'name': '1', 'data': 1}
{'name': '2', 'data': 2}
{'name': '3', 'data': 3}
{'name': '4', 'data': 4}


In [3]:
squares = ds.map(lambda x: x["data"] ** 2)  

evens = squares.filter(lambda x: x % 2 == 0)
evens.count()

cubes = evens.flat_map(lambda x: [x, x**3])  
sample = cubes.take(10)  
print(sample)

[dataset]: Run `pip install tqdm` to enable progress reporting.
[0, 0, 4, 64, 16, 4096, 36, 46656, 64, 262144]


In [4]:
# tag::ray_data_pipeline[]
pipe = ds.pipeline()  # <1>
result = pipe\
    .map(lambda x: x["data"] ** 2)\
    .filter(lambda x: x % 2 == 0)\
    .flat_map(lambda x: [x, x**3])  # <2>
result.show(10)
# end::ray_data_pipeline[]

AttributeError: 'Dataset' object has no attribute 'pipeline'

In [1]:
# tag::ray_tune[]
from ray import tune
import math
import time


def training_function(config):  # <1>
    x, y = config["x"], config["y"]
    time.sleep(10)
    score = objective(x, y)
    tune.report(score=score)  # <2>


def objective(x, y):
    return math.sqrt((x**2 + y**2)/2)  # <3>


result = tune.run(  # <4>
    training_function,
    config={
        "x": tune.grid_search([-1, -.5, 0, .5, 1]),  # <5>
        "y": tune.grid_search([-1, -.5, 0, .5, 1])
    })

print(result.get_best_config(metric="score", mode="min"))
# end::ray_tune[]

2022-04-30 11:24:20,788	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8266
2022-04-30 11:24:24,269	WARNING function_runner.py:598 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-04-30 11:24:24,858	INFO trial_runner.py:803 -- starting training_function_5337a_00000


Trial name,status,loc,x,y
training_function_5337a_00000,RUNNING,192.168.2.133:2124,-1,-1
training_function_5337a_00001,PENDING,,-0.5,-1
training_function_5337a_00002,PENDING,,0,-1
training_function_5337a_00003,PENDING,,0.5,-1
training_function_5337a_00004,PENDING,,1,-1
training_function_5337a_00005,PENDING,,-1,-0.5
training_function_5337a_00006,PENDING,,-0.5,-0.5
training_function_5337a_00007,PENDING,,0,-0.5
training_function_5337a_00008,PENDING,,0.5,-0.5
training_function_5337a_00009,PENDING,,1,-0.5


Trial name,status,loc,x,y
training_function_5337a_00000,RUNNING,192.168.2.133:2124,-1,-1
training_function_5337a_00001,PENDING,,-0.5,-1
training_function_5337a_00002,PENDING,,0,-1
training_function_5337a_00003,PENDING,,0.5,-1
training_function_5337a_00004,PENDING,,1,-1
training_function_5337a_00005,PENDING,,-1,-0.5
training_function_5337a_00006,PENDING,,-0.5,-0.5
training_function_5337a_00007,PENDING,,0,-0.5
training_function_5337a_00008,PENDING,,0.5,-0.5
training_function_5337a_00009,PENDING,,1,-0.5


Result for training_function_5337a_00000:
  date: 2022-04-30_11-24-41
  done: false
  experiment_id: e651eec37be24a68be1d7c0bc020edcd
  hostname: patrick-c3-1
  iterations_since_restore: 1
  node_ip: 192.168.2.133
  pid: 2124
  score: 1.0
  time_since_restore: 10.019779205322266
  time_this_iter_s: 10.019779205322266
  time_total_s: 10.019779205322266
  timestamp: 1651310681
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5337a_00000
  warmup_time: 0.03398442268371582
  


Trial name,status,loc,x,y,iter,total time (s),score
training_function_5337a_00000,RUNNING,192.168.2.133:2124,-1,-1,1,10.0198,1
training_function_5337a_00001,PENDING,,-0.5,-1,,,
training_function_5337a_00002,PENDING,,0,-1,,,
training_function_5337a_00003,PENDING,,0.5,-1,,,
training_function_5337a_00004,PENDING,,1,-1,,,
training_function_5337a_00005,PENDING,,-1,-0.5,,,
training_function_5337a_00006,PENDING,,-0.5,-0.5,,,
training_function_5337a_00007,PENDING,,0,-0.5,,,
training_function_5337a_00008,PENDING,,0.5,-0.5,,,
training_function_5337a_00009,PENDING,,1,-0.5,,,


...

Trial name,status,loc,x,y,iter,total time (s),score
training_function_5337a_00024,RUNNING,192.168.2.133:3587,1,1,,,
training_function_5337a_00000,TERMINATED,192.168.2.133:2124,-1,-1,1,10.0198,1
training_function_5337a_00001,TERMINATED,192.168.2.133:2185,-0.5,-1,1,10.0095,0.790569
training_function_5337a_00002,TERMINATED,192.168.2.133:2247,0,-1,1,10.0104,0.707107
training_function_5337a_00003,TERMINATED,192.168.2.133:2308,0.5,-1,1,10.0092,0.790569
training_function_5337a_00004,TERMINATED,192.168.2.133:2369,1,-1,1,10.0141,1
training_function_5337a_00005,TERMINATED,192.168.2.133:2428,-1,-0.5,1,10.0114,0.790569
training_function_5337a_00006,TERMINATED,192.168.2.133:2489,-0.5,-0.5,1,10.0104,0.5
training_function_5337a_00007,TERMINATED,192.168.2.133:2551,0,-0.5,1,10.0056,0.353553
training_function_5337a_00008,TERMINATED,192.168.2.133:2612,0.5,-0.5,1,10.0102,0.5


Result for training_function_5337a_00024:
  date: 2022-04-30_11-30-17
  done: false
  experiment_id: aa3964745d704baf98fea804cc60937c
  hostname: patrick-c3-1
  iterations_since_restore: 1
  node_ip: 192.168.2.133
  pid: 3587
  score: 1.0
  time_since_restore: 10.009488344192505
  time_this_iter_s: 10.009488344192505
  time_total_s: 10.009488344192505
  timestamp: 1651311017
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5337a_00024
  warmup_time: 0.0058135986328125
  
Result for training_function_5337a_00024:
  date: 2022-04-30_11-30-17
  done: true
  experiment_id: aa3964745d704baf98fea804cc60937c
  experiment_tag: 24_x=1,y=1
  hostname: patrick-c3-1
  iterations_since_restore: 1
  node_ip: 192.168.2.133
  pid: 3587
  score: 1.0
  time_since_restore: 10.009488344192505
  time_this_iter_s: 10.009488344192505
  time_total_s: 10.009488344192505
  timestamp: 1651311017
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5337a_00024
  warmup_time: 0.00581359

Trial name,status,loc,x,y,iter,total time (s),score
training_function_5337a_00000,TERMINATED,192.168.2.133:2124,-1,-1,1,10.0198,1
training_function_5337a_00001,TERMINATED,192.168.2.133:2185,-0.5,-1,1,10.0095,0.790569
training_function_5337a_00002,TERMINATED,192.168.2.133:2247,0,-1,1,10.0104,0.707107
training_function_5337a_00003,TERMINATED,192.168.2.133:2308,0.5,-1,1,10.0092,0.790569
training_function_5337a_00004,TERMINATED,192.168.2.133:2369,1,-1,1,10.0141,1
training_function_5337a_00005,TERMINATED,192.168.2.133:2428,-1,-0.5,1,10.0114,0.790569
training_function_5337a_00006,TERMINATED,192.168.2.133:2489,-0.5,-0.5,1,10.0104,0.5
training_function_5337a_00007,TERMINATED,192.168.2.133:2551,0,-0.5,1,10.0056,0.353553
training_function_5337a_00008,TERMINATED,192.168.2.133:2612,0.5,-0.5,1,10.0102,0.5
training_function_5337a_00009,TERMINATED,192.168.2.133:2672,1,-0.5,1,10.0087,0.790569


2022-04-30 11:30:17,400	INFO tune.py:701 -- Total run time: 353.14 seconds (352.66 seconds for the tuning loop).


{'x': 0, 'y': 0}
